TITANIC SURVIVOR ANALYSIS

In [1]:
import pandas as pd
import numpy as np

READING DATA USING PANDAS

We use pandas read_csv function to read the csv file in python and pandas DataFrame method to convert file into the dataframe.

In [4]:
df=pd.DataFrame(pd.read_csv('/content/train (1).csv'))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.shape

(891, 12)

HANDLING NULL VALUES

The dataset may contain rows and columns with missing values, so we can either drop the entire row or column, OR fill the missing values with 
some appropriate value, example: mean of all values for a particular column

In [6]:
df.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

SEPARATING OUT COLUMNS WHICH HAVE MORE THAN 35% OF THE VALUES MISSING IN THE DATASET

In [8]:
x=df.isnull().sum()
drop_col=x[x>(35/100 * df.shape[0])]
drop_col

Cabin    687
dtype: int64

There's no specific number after which we should drop the column. Its just we decided on our own according to our preferences what we want

In [9]:
drop_col.index

Index(['Cabin'], dtype='object')

In [10]:
df.drop(drop_col.index,axis=1,inplace=True)
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [11]:
df.fillna(df.mean(),inplace=True)
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

In [12]:
df['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

For Embarked attribute, we fill the NULL values with the most frequent value in the column.

In [13]:
df['Embarked'].fillna('S',inplace=True)
df.isnull().sum()


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [14]:
df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


Here through the above correlation, we can conclude that Fare and Pclass have a good correlation, although negative.

We can make a new column family_size by combining these two columns.

In [15]:
df['FamilySize']=df['SibSp']+df['Parch']
df.drop(['SibSp','Parch'], axis=1,inplace=True)
df.corr()

,PassengerId,Survived,Pclass,Age,Fare,FamilySize
PassengerId,1.000000,-0.005007,-0.035144,0.033207,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,0.091566,-0.248512
Fare,0.012658,0.257307,-0.549500,0.091566,1.000000,0.217138
FamilySize,-0.040143,0.016639,0.065997,-0.248512,0.217138,1.000000


FamilySize in the ship does not have much correlation with the survival rate, whereas the Pclass has a good negative correlation with Survived 

Check whether the person was alone or not can affect the survival rate.

In [16]:
df['Alone']=[0 if df['FamilySize'][i]>0 else 1 for i in df.index]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,FamilySize,Alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,0,1


In [17]:
df.groupby(['Alone'])['Survived'].mean()

Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

We can conclude from above that a person's survival rate is higher when he is not alone. Since mean value is much higher when Alone column is 0. If the person is alone, he/she has a lesser chance of survival.

The reason might be the person who is traveling with his family might be belonging to rich class and so, might be prioritized over other.

In [18]:
df[['Alone','Fare']].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


We can conclude that if the person is not alone, the chance that ticket price is higher is high.

In [19]:
df['Sex']=[0 if df['Sex'][i]=='male' else 1 for i in df.index] #1 for female, 0 for male
df.groupby(['Sex'])['Survived'].mean()

Sex
0    0.188908
1    0.742038
Name: Survived, dtype: float64

Here, through above output, we can conclude that female passengers have more chance of surviving than male ones. Women were prioritized over men.

In [20]:
df.groupby(['Embarked'])['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

We can conclude that percentage of survival was more for people with Embarkation port as C, whereas it was minimum for people with Embarkation port as S.

CONCLUSION

1. Female passengers were prioritized over men.
2. People with high class or rich people have higher survival rate than others. The hierarchy might have been followed while saving the passengers.
3. Passengers travelling with their family have higher survival rate than those travelling alone.
4. Passengers who boarded the ship at Cherbourg, survived more in proportion than the others.